In [1]:
# data wrangling and merging for Project 3, BIOS 611

# load required packages
import pandas as pd
import numpy as np
import datetime as datetime
import time
import calendar

# read in UMD shelter source data from github
demog_orig = pd.read_csv('https://raw.githubusercontent.com/biodatascience/datasci611/gh-pages/data/project2_2019/CLIENT_191102.tsv',sep='\t')
visits_orig = pd.read_csv('https://raw.githubusercontent.com/biodatascience/datasci611/gh-pages/data/project2_2019/ENTRY_EXIT_191102.tsv',sep='\t')
survey_orig = pd.read_csv('https://raw.githubusercontent.com/biodatascience/datasci611/gh-pages/data/project2_2019/EE_UDES_191102.tsv',sep='\t')

#visits_orig.head()
#list(visits_orig.columns)

In [5]:

# date formatting from visits data, want to split up months, years, use numeric for time differences and sorting
visits=visits_orig

# calculate number of days stayed for each person
visits['first_datenum'] =  pd.to_datetime(visits['Entry Date'], format='%m/%d/%Y')
visits['last_datenum'] =  pd.to_datetime(visits['Exit Date'], format='%m/%d/%Y')
visits['diffdays'] = visits['last_datenum'] - visits['first_datenum']  # difference automatically calculated in days
visits['diffdays'] = visits['diffdays'] / np.timedelta64(1, 'D') # want as numeric

# extract month as numeric, month as character, and year
visits['first_m'] = pd.DatetimeIndex(visits['Entry Date']).month
visits['last_m'] = pd.DatetimeIndex(visits['Exit Date']).month
#visits['first_mc'] = visits['first_m'].apply(lambda x: calendar.month_abbr[x])
#visits['last_mc'] = visits['last_m'].apply(lambda x: calendar.month_abbr[x])
visits['first_year'] = pd.DatetimeIndex(visits['Entry Date']).year
visits['last_year'] = pd.DatetimeIndex(visits['Exit Date']).year

# keep necessary columns: EE UID, client ID, entry date, exit date, diffdays, months, years
visits_sub = visits[['EE UID','Client ID','Destination','first_datenum','last_datenum','first_m','last_m','first_year','last_year']]

visits_sub.head()

,EE UID,Client ID,Destination,first_datenum,last_datenum,first_m,last_m,first_year,last_year
0,687901,397941,"Rental by client, with other ongoing housing s...",2015-08-15,2016-07-11,8,7.0,2015,2016.0
1,687902,130335,Data not collected (HUD),2015-08-15,2015-08-31,8,8.0,2015,2015.0
2,687903,188933,"Staying or living with friends, temporary tenu...",2015-08-15,2015-09-19,8,9.0,2015,2015.0
3,687904,168290,Hospital or other residential non-psychiatric ...,2015-08-15,2016-03-07,8,3.0,2015,2016.0
4,687905,123122,"Staying or living with friends, temporary tenu...",2015-08-15,2015-08-24,8,8.0,2015,2015.0


In [11]:
## processing survey data 
survey = survey_orig
print(survey.columns)
survey_sub = survey[['EE UID','Client ID','Length of Stay in Previous Place(1934)','Housing Status(2703)']]
survey_sub.head()

Index(['EE Provider ID', 'Entry Exit Provider Program Type Code', 'EE UID',
       'Client Unique ID', 'Client ID', 'Client Location(4378)',
       'Zip Code (of Last Permanent Address, if known)(1932)',
       'Relationship to Head of Household(4374)', 'Prior Living Situation(43)',
       'Length of Stay in Previous Place(1934)',
       'Did you stay less than 7 nights?(5164)',
       'Did you stay less than 90 days?(5163)',
       'On the night before did you stay on the streets, ES or SH?(5165)',
       'Regardless of where they stayed last night - Number of times the client has been on the streets, in ES, or SH in the past three years including today(5167)',
       'Total number of months homeless on the street, in ES or SH in the past three years(5168)',
       'Housing Status(2703)',
       'Does the client have a disabling condition?(1935)',
       'Covered by Health Insurance(4376)',
       'Domestic violence victim/survivor(341)',
       'If yes for Domestic violence victim/su

,EE UID,Client ID,Length of Stay in Previous Place(1934),Housing Status(2703)
0,687901,397941,One year or longer (HUD),Category 1 - Homeless (HUD)
1,687902,130335,"One month or more, but less than 90 days",Category 1 - Homeless (HUD)
2,687903,188933,"One month or more, but less than 90 days",Category 1 - Homeless (HUD)
3,687904,168290,One year or longer (HUD),Category 1 - Homeless (HUD)
4,687905,123122,"One week or more, but less than one month",Category 1 - Homeless (HUD)


In [17]:
# subset demog data
demog = demog_orig
print(demog.columns)
demog_sub = demog[['EE UID','Client ID','Client Age at Entry','Client Gender','Client Gender','Client Primary Race', 'Client Ethnicity', 'Client Veteran Status']]

Index(['EE Provider ID', 'EE UID', 'Client Unique ID', 'Client ID',
       'Client Age at Entry', 'Client Age at Exit', 'Client Gender',
       'Client Primary Race', 'Client Ethnicity', 'Client Veteran Status'],
      dtype='object')


In [19]:
# merge survey, visits, and demog to address the following questions graphically: 
# is length of stay in previous place befofere umd associated with how long they stayed at umd?
# is housing status associated with how long people stay at UMD?
# does client growth differ over time by gender, or do men/women/nonbinary arrive at same rate?
# include other demog visuals
# are some months more popular than others for coming and going?

dat = visits_sub.merge(demog_sub, on=['EE UID','Client ID'], how='left')
dat = dat.merge(survey_sub,on=['EE UID','Client ID'],how ='left')
dat.head()




,EE UID,Client ID,Destination,first_datenum,last_datenum,first_m,last_m,first_year,last_year,Client Age at Entry,Client Gender,Client Gender,Client Primary Race,Client Ethnicity,Client Veteran Status,Length of Stay in Previous Place(1934),Housing Status(2703)
0,687901,397941,"Rental by client, with other ongoing housing s...",2015-08-15,2016-07-11,8,7.0,2015,2016.0,60.0,Female,Female,White (HUD),Non-Hispanic/Non-Latino (HUD),No (HUD),One year or longer (HUD),Category 1 - Homeless (HUD)
1,687902,130335,Data not collected (HUD),2015-08-15,2015-08-31,8,8.0,2015,2015.0,48.0,Female,Female,Black or African American (HUD),Non-Hispanic/Non-Latino (HUD),No (HUD),"One month or more, but less than 90 days",Category 1 - Homeless (HUD)
2,687903,188933,"Staying or living with friends, temporary tenu...",2015-08-15,2015-09-19,8,9.0,2015,2015.0,42.0,Female,Female,Black or African American (HUD),Non-Hispanic/Non-Latino (HUD),No (HUD),"One month or more, but less than 90 days",Category 1 - Homeless (HUD)
3,687904,168290,Hospital or other residential non-psychiatric ...,2015-08-15,2016-03-07,8,3.0,2015,2016.0,57.0,Male,Male,White (HUD),Hispanic/Latino (HUD),No (HUD),One year or longer (HUD),Category 1 - Homeless (HUD)
4,687905,123122,"Staying or living with friends, temporary tenu...",2015-08-15,2015-08-24,8,8.0,2015,2015.0,51.0,Male,Male,White (HUD),Non-Hispanic/Non-Latino (HUD),No (HUD),"One week or more, but less than one month",Category 1 - Homeless (HUD)


In [21]:
# export dataset with visits, demographics, and survey results to .csv

export_csv = dat.to_csv (r'../data/p3_clean.csv', index = None, header=True)